In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import necessary libraries

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer, LancasterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re, string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.linear_model import LogisticRegression, SGDClassifier, Ridge
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, f1_score
import os
from sklearn.ensemble import RandomForestClassifier
from scipy.linalg import inv
from numpy.linalg import pinv

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticate with Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset
dataset = 'lakshmi25npathi/imdb-dataset-of-50k-movie-reviews'
path = '.'
api.dataset_download_files(dataset, path=path, unzip=True)

print("Dataset downloaded and extracted to:", path)

In [ ]:
# Load the IMDB dataset directly from Kaggle's dataset path
imdb_data = pd.read_csv('IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

In [ ]:
# Summary of the dataset
imdb_data.describe()

In [ ]:
#sentiment count
imdb_data['sentiment'].value_counts()

In [ ]:
#split the dataset  
#train dataset
train_reviews=imdb_data.review[:40000]
train_sentiments=imdb_data.sentiment[:40000]
#test dataset
test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [ ]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [ ]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

In [ ]:
#normalized train reviews
norm_train_reviews=imdb_data.review[:40000]
norm_train_reviews[0]
#convert dataframe to string
#norm_train_string=norm_train_reviews.to_string()
#Spelling correction using Textblob
#norm_train_spelling=TextBlob(norm_train_string)
#norm_train_spelling.correct()
#Tokenization using Textblob
#norm_train_words=norm_train_spelling.words
#norm_train_words

In [ ]:
#Normalized test reviews
norm_test_reviews=imdb_data.review[40000:]
norm_test_reviews[45005]
##convert dataframe to string
#norm_test_string=norm_test_reviews.to_string()
#spelling correction using Textblob
#norm_test_spelling=TextBlob(norm_test_string)
#print(norm_test_spelling.correct())
#Tokenization using Textblob
#norm_test_words=norm_test_spelling.words
#norm_test_words

In [ ]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=1,max_df=1.0,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

In [ ]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=1,max_df=1.0,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

In [ ]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

In [ ]:
#Spliting the sentiment data
train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]
print(train_sentiments)
print(test_sentiments)

## Training the RVFL Model

In [ ]:
# Model training and evaluation code (assuming preprocessed data is already available)

# RVFL model definition
class RVFL:
    def __init__(self, n_nodes=100, random_weight_range=(-1, 1), random_bias_range=(-1, 1), regularization=0.01):
        self.n_nodes = n_nodes
        self.random_weight_range = random_weight_range
        self.random_bias_range = random_bias_range
        self.regularization = regularization
        self.input_weights = None
        self.biases = None
        self.output_weights = None
        self.scaler = StandardScaler(with_mean=False)
    
    def _activation(self, X):
        """ReLU activation function."""
        return np.maximum(0, X)

    def fit(self, X_train, y_train):
        X_train = self.scaler.fit_transform(X_train)
        self.input_weights = np.random.uniform(self.random_weight_range[0], self.random_weight_range[1], (X_train.shape[1], self.n_nodes))
        self.biases = np.random.uniform(self.random_bias_range[0], self.random_bias_range[1], self.n_nodes)
        H = self._activation(np.dot(X_train, self.input_weights) + self.biases)
        H_augmented = np.hstack((X_train, H))
        ridge_regressor = Ridge(alpha=self.regularization, fit_intercept=False)
        ridge_regressor.fit(H_augmented, y_train)
        self.output_weights = ridge_regressor.coef_

    def predict(self, X_test):
        X_test = self.scaler.transform(X_test)
        H = self._activation(np.dot(X_test, self.input_weights) + self.biases)
        H_augmented = np.hstack((X_test, H))
        y_pred = np.dot(H_augmented, self.output_weights)
        return (y_pred >= 0.5).astype(int)

# Assuming preprocessed and vectorized training and test data are available as:
# cv_train_reviews, train_sentiments, cv_test_reviews, test_sentiments

# Train and evaluate the RVFL model
rvfl_model = RVFL(n_nodes=100, random_weight_range=(-1, 1), random_bias_range=(-1, 1), regularization=0.01)
rvfl_model.fit(cv_train_reviews, train_sentiments)
print("Model training complete!")

y_pred = rvfl_model.predict(cv_test_reviews)

# Evaluate performance
accuracy = accuracy_score(test_sentiments, y_pred)
print(f'Accuracy: {accuracy:.4f}')

auc = roc_auc_score(test_sentiments, y_pred)
print(f'AUC-ROC: {auc:.4f}')

gmean = geometric_mean_score(test_sentiments, y_pred)
print(f'G-Mean: {gmean:.4f}')

cm = confusion_matrix(test_sentiments, y_pred)
print("Confusion Matrix:")
print(cm)

# Plot confusion matrix
def plot_confusion_matrix(cm, labels):
    fig, ax = plt.subplots()
    cax = ax.matshow(cm, cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + labels)
    ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

plot_confusion_matrix(cm, ['Class 0', 'Class 1'])

## Training the Weighted RVFL Model

In [ ]:
# Data scaling
scaler = StandardScaler(with_mean=False)  # with_mean=False because sparse matrices do not support mean centering
x_train_scaled = scaler.fit_transform(cv_train_reviews)
x_val_scaled = scaler.transform(cv_test_reviews)

# Define a function to calculate G-Mean
def g_mean(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    return np.sqrt(sensitivity * specificity)

# Weighted RVFL model (initial setup with RandomForest for comparison)
weighted_rvfl_model = RandomForestClassifier(class_weight="balanced", n_estimators=100, random_state=42)
weighted_rvfl_model.fit(x_train_scaled, train_sentiments.ravel())

# Predictions and evaluation
y_pred = weighted_rvfl_model.predict(x_val_scaled)
y_prob = weighted_rvfl_model.predict_proba(x_val_scaled)[:, 1]

# Calculate evaluation metrics
auc = roc_auc_score(test_sentiments, y_prob)
f1 = f1_score(test_sentiments, y_pred)
accuracy = accuracy_score(test_sentiments, y_pred)
gmean = g_mean(test_sentiments, y_pred)

print("Weighted RVFL Performance Metrics:")
print(f"AUC-ROC: {auc:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"G-Mean: {gmean:.4f}")

## Training the Boosting Weighted RVFL Model

In [ ]:
# Scale the data
scaler = StandardScaler(with_mean=False)  # with_mean=False for sparse matrices
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# RVFL class with Ridge regression
class RVFL:
    def __init__(self, input_size, hidden_nodes=100, activation_function='relu', ridge_lambda=1.0):
        self.input_size = input_size
        self.hidden_nodes = hidden_nodes
        self.activation_function = activation_function
        self.ridge_lambda = ridge_lambda
        self.random_weights = np.random.randn(input_size, hidden_nodes)
        self.random_biases = np.random.randn(hidden_nodes)

    def relu(self, x):
        return np.maximum(0, x)

    def leaky_relu(self, x):
        return np.where(x > 0, x, x * 0.01)

    def transform(self, X):
        H = np.dot(X, self.random_weights) + self.random_biases
        if self.activation_function == 'relu':
            H = self.relu(H)
        elif self.activation_function == 'leaky_relu':
            H = self.leaky_relu(H)
        return np.hstack((X.toarray(), H))  # Convert sparse to dense before stacking

    def fit(self, X_train, y_train):
        H_train = self.transform(X_train)
        self.scaler = StandardScaler().fit(H_train)
        H_train = self.scaler.transform(H_train)
        self.ridge_reg = Ridge(alpha=self.ridge_lambda)
        self.ridge_reg.fit(H_train, y_train)

    def predict(self, X_test):
        H_test = self.transform(X_test)
        H_test = self.scaler.transform(H_test)
        return self.ridge_reg.predict(H_test)

    def predict_classes(self, X_test):
        return np.where(self.predict(X_test) > 0.5, 1, 0)

# Boosting Weighted RVFL with learning rate
class BoostingWeightedRVFL:
    def __init__(self, num_models=10, hidden_nodes=100, learning_rate=0.1):
        self.num_models = num_models
        self.models = []
        self.model_weights = []
        self.hidden_nodes = hidden_nodes
        self.learning_rate = learning_rate

    def fit(self, X_train, y_train):
        N = X_train.shape[0]
        weights = np.ones(N) / N

        for i in range(self.num_models):
            model = RVFL(input_size=X_train.shape[1], hidden_nodes=self.hidden_nodes)
            model.fit(X_train, y_train)
            y_pred = model.predict_classes(X_train)

            errors = np.abs(y_train - y_pred)
            weighted_error = np.dot(weights, errors) / np.sum(weights)

            if weighted_error < 1e-10:
                break

            alpha = self.learning_rate * np.log((1 - weighted_error) / (weighted_error + 1e-10))
            self.models.append(model)
            self.model_weights.append(alpha)

            weights = weights * np.exp(alpha * errors)
            weights /= np.sum(weights)

    def predict(self, X_test):
        final_predictions = np.zeros(X_test.shape[0])
        for model, alpha in zip(self.models, self.model_weights):
            final_predictions += alpha * model.predict_classes(X_test)
        return np.where(final_predictions > 0, 1, 0)

# Manual G-Mean calculation
def calculate_gmean(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    gmean = np.sqrt(sensitivity * specificity)
    return gmean

# Training and evaluation
model = BoostingWeightedRVFL(num_models=5, hidden_nodes=50, learning_rate=0.001)
model.fit(x_train_scaled, y_train)

y_pred = model.predict(x_test_scaled)

auc_roc = roc_auc_score(y_test, y_pred)
gmean = calculate_gmean(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("AUC-ROC Score:", auc_roc)
print("G-Mean:", gmean)
print("Accuracy:", accuracy)

## Training the Inception V3 Model

In [ ]:
# Define G-Mean function for evaluation
def g_mean(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    return np.sqrt(sensitivity * specificity)

# Ensure processed data is already in the notebook environment:
# tv_train_reviews, tv_test_reviews, train_sentiments, test_sentiments

# Convert sparse matrices to dense arrays if needed
tv_train_reviews = tv_train_reviews.toarray()
tv_test_reviews = tv_test_reviews.toarray()

# Model architecture for text data
input_layer = Input(shape=(tv_train_reviews.shape[1],))

# Multi-layer dense network
tower_1 = Dense(128, activation="relu")(input_layer)
tower_1 = Dropout(0.3)(tower_1)

tower_2 = Dense(64, activation="relu")(input_layer)
tower_2 = Dropout(0.3)(tower_2)

tower_3 = Dense(32, activation="relu")(input_layer)
tower_3 = Dropout(0.3)(tower_3)

# Concatenate the layers
concatenated = Concatenate()([tower_1, tower_2, tower_3])

# Output layer for binary classification
output_layer = Dense(1, activation="sigmoid")(concatenated)

# Compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model.fit(
    tv_train_reviews, train_sentiments,
    epochs=50,
    batch_size=64,
    validation_split=0.2,  # Use part of training data for validation
    callbacks=[early_stopping],
    verbose=1
)

# Predictions and evaluation
y_pred_probs = model.predict(tv_test_reviews).flatten()  # Sigmoid outputs
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_probs]

# Evaluation metrics
auc = roc_auc_score(test_sentiments, y_pred_probs)
f1 = f1_score(test_sentiments, y_pred)
accuracy = accuracy_score(test_sentiments, y_pred)
gmean = g_mean(test_sentiments, y_pred)

print("Model Performance Metrics:")
print(f"AUC-ROC: {auc:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"G-Mean: {gmean:.4f}")

## Training the SVM Model

In [ ]:
# Assuming preprocessed data is available in the notebook from prior steps
# Extract data from previously defined variables
x_train = cv_train_reviews  # Replace with actual variable if different
x_test = cv_test_reviews    # Replace with actual variable if different
y_train = train_sentiments.ravel()  # Flatten y
y_test = test_sentiments.ravel()    # Flatten y

# Scale the data
scaler = StandardScaler(with_mean=False)  # Set with_mean=False to handle sparse matrices
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Define a function to calculate G-Mean
def g_mean(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sensitivity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    return np.sqrt(sensitivity * specificity)

# Initialize and train the SVM model with class weights to address imbalance
svm_model = SVC(kernel="linear", class_weight="balanced", probability=True, random_state=42)
svm_model.fit(x_train_scaled, y_train)

# Predictions and evaluation
y_pred = svm_model.predict(x_test_scaled)
y_prob = svm_model.predict_proba(x_test_scaled)[:, 1]

# Calculate evaluation metrics
auc = roc_auc_score(y_test, y_prob)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
gmean = g_mean(y_test, y_pred)

# Print out the performance metrics
print("SVM Performance Metrics:")
print(f"AUC-ROC: {auc:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"G-Mean: {gmean:.4f}")

## Training the AUC-Based RVFL Model

In [ ]:
# Assume x_train, x_test, y_train, y_test have been preloaded in the same notebook

# Define RVFL with Iterative AUC optimization
class IterativeAUC_RVFL:
    def __init__(self, n_hidden_neurons=100, gamma=0.01, max_iters=100, tol=1e-4):
        self.n_hidden_neurons = n_hidden_neurons
        self.gamma = gamma
        self.max_iters = max_iters
        self.tol = tol  # Tolerance for convergence

    def initialize_weights_and_biases(self, n_features):
        # Random weight and bias initialization
        self.input_weights = np.random.randn(n_features, self.n_hidden_neurons)
        self.biases = np.random.randn(self.n_hidden_neurons)

    def hidden_layer_output(self, X):
        # Calculate hidden layer output using ReLU activation
        H = np.dot(X, self.input_weights) + self.biases
        return np.maximum(H, 0)  # ReLU

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.initialize_weights_and_biases(n_features)

        # Generate initial hidden layer output matrix H
        H = self.hidden_layer_output(X)
        
        # Initial computation for matrices A and Y
        A = np.zeros((self.n_hidden_neurons, self.n_hidden_neurons))
        Y = np.zeros((self.n_hidden_neurons, 1))
        for i in range(n_samples):
            if y[i] == 1:  # Positive class
                A += np.outer(H[i], H[i])
                Y += H[i].reshape(-1, 1)
            else:  # Negative class
                A -= np.outer(H[i], H[i])
                Y -= H[i].reshape(-1, 1)
        
        # Initialize beta with zeros
        self.beta = np.zeros((self.n_hidden_neurons, 1))
        
        # Iterative training to update beta
        for iteration in range(self.max_iters):
            # Compute output weights beta with iterative improvement
            if n_samples > self.n_hidden_neurons:
                beta_new = inv((n_samples / self.gamma) * A + np.dot(H.T, H)).dot(H.T).dot(Y)
            else:
                beta_new = H.T.dot(inv(np.dot(H, H.T) + (n_samples / self.gamma) * A)).dot(Y)
            
            # Check for convergence based on tolerance
            if np.linalg.norm(self.beta - beta_new) < self.tol:
                print(f"Converged after {iteration + 1} iterations")
                break
            
            # Update beta for the next iteration
            self.beta = beta_new
        
        # Final beta after iteration
        self.beta = beta_new

    def predict(self, X):
        # Calculate hidden layer output and final predictions
        H = self.hidden_layer_output(X)
        output = np.dot(H, self.beta)
        return np.where(output >= 0.5, 1, 0)

# Instantiate and Train Iterative AUC-based RVFL
model = IterativeAUC_RVFL(n_hidden_neurons=100, gamma=0.01, max_iters=100, tol=1e-4)
model.fit(x_train, y_train)

# Predict and Evaluate
y_pred = model.predict(x_test)
auc_score = roc_auc_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
g_mean = np.sqrt(confusion_matrix(y_test, y_pred).diagonal().prod() / (y_test.sum() * (len(y_test) - y_test.sum())))

print(f"Iterative AUC-RVFL Performance Metrics:\n"
      f"AUC-ROC Score: {auc_score:.4f}\n"
      f"Accuracy: {accuracy:.4f}\n"
      f"F1 Score: {f1:.4f}\n"
      f"G-Mean: {g_mean:.4f}")